<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/pooling_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [2]:
import numpy as np
import pandas as pd

from ddop.newsvendor import RandomForestWeightedNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from sklearn.model_selection import RandomizedSearchCV

from ddop.metrics import average_costs
from ddop.metrics import prescriptiveness_score
from ddop.metrics import make_scorer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## loading data-set


---



*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**

In [4]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## split in feature and target matrix


---

In [5]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split


---

In [6]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## new order after the split


---

*   for the product pooling the whole data must be be sorted by date independent from the 'item_id'
*   therefore sort values by 'date' in the first instance and by 'item_id' in the second
*   these two instances create the mutli-index of the target matrix (Y)
*   as all items are in one set now the 'item_id' becomes a feature in the feature matrix (X). The only index is the date here.

In [7]:
X_train = X_train.reset_index().sort_values(by=['date', 'item_id']).set_index(['date'])
X_test = X_test.reset_index().sort_values(by=['date', 'item_id']).set_index(['date'])

Y_train = Y_train.reset_index().sort_values(by=['date', 'item_id']).set_index(['date', 'item_id'])
Y_test = Y_test.reset_index().sort_values(by=['date', 'item_id']).set_index(['date', 'item_id'])

## defining the different cost paramerters for special service level and product range



---



*   cu is the mean of the cumulated 'sell_price' column of the training data
*   appending to the corresponding service level co is calculated with (cu - α * cu) / α
*   all the costs are rounded to 2 digits

In [8]:
prods = list(range(0,25))

cu = round(X_train['sell_price'].mean(),2)

co_90 = round(((cu-(0.90*cu))/0.90),2)

co_75 = round(((cu-(0.75*cu))/0.75),2)

co_50 = round(((cu-(0.50*cu))/0.50),2)

## determining some parameters


---


*   the potential range of the params for randomized search are defined
*   the estimator, the scorer and finally the whole param_random is defined

In [17]:
max_depth = np.arange(7, 24)
n_estimators = np.arange(65, 101)
min_samples_leaf = np.arange(2, 6)
param_random = dict(max_depth = max_depth, n_estimators = n_estimators, min_samples_leaf = min_samples_leaf)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

# rf for 90% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: n_estimators = 79 , max_depth = 20 , min_samples_leaf = 4

In [ ]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse')
random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 90%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [9]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse',
                                    max_depth = 20, n_estimators = 79, min_samples_leaf = 4)

rf.fit(X_train, Y_train)
preds_90 = rf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ddop/newsvendor/_WeightedNewsvendor.py:580: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


## reorder the data




---



*   create a pandas frame 'comp_90' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [10]:
pred_90 = pd.DataFrame(preds_90, columns=['pred_rf_90'])
comp_90 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_90 = comp_90.reset_index()
comp_90['pred_rf_90'] = pred_90[['pred_rf_90']]
comp_90 = comp_90.sort_values(by=['item_id', 'date']).set_index(['item_id', 'date'])

saa_90 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_90.csv').set_index(['item_id', 'date'])

comp_90 = pd.concat([comp_90, saa_90], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame


In [12]:
result_rf_90_avc = []

for prod in prods:
  avc = round(average_costs(comp_90['demand'].loc[prod], comp_90['pred_rf_90'].loc[prod], cu = cu, co = co_90),2)
  result_rf_90_avc.append(avc)

In [13]:
RF_pooled_avc_90 = pd.DataFrame(result_rf_90_avc, columns=['RF_avc_90'])
RF_pooled_avc_90.index.name = 'item_id'

In [15]:
result_rf_90_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_90['demand'].loc[prod], comp_90['pred_rf_90'].loc[prod],
                                      comp_90['SAA_pred_90'].loc[prod], cu = cu, co = co_90),2)
  result_rf_90_pscr.append(pscr)

In [16]:
RF_pooled_pscr_90 = pd.DataFrame(result_rf_90_pscr, columns=['RF_pscr_90'])
RF_pooled_pscr_90.index.name = 'item_id'

# rf for 75% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: n_estimators = 78 , max_depth = 19 , min_samples_leaf = 3

In [ ]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse')
random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 75%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [18]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse',
                                    max_depth = 19, n_estimators = 78, min_samples_leaf = 3)

rf.fit(X_train, Y_train)
preds_75 = rf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ddop/newsvendor/_WeightedNewsvendor.py:580: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


## reorder the data




---



*   create a pandas frame 'comp_75' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [19]:
pred_75 = pd.DataFrame(preds_75, columns=['pred_rf_75'])
comp_75 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_75 = comp_75.reset_index()
comp_75['pred_rf_75'] = pred_75[['pred_rf_75']]
comp_75 = comp_75.set_index(['item_id', 'date'])

saa_75 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_75.csv').set_index(['item_id', 'date'])

comp_75 = pd.concat([comp_75, saa_75], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [20]:
result_rf_75_avc = []

for prod in prods:
  avc = round(average_costs(comp_75['demand'].loc[prod], comp_75['pred_rf_75'].loc[prod], cu = cu, co = co_75),2)
  result_rf_75_avc.append(avc)

In [21]:
RF_pooled_avc_75 = pd.DataFrame(result_rf_75_avc, columns=['RF_avc_75'])
RF_pooled_avc_75.index.name = 'item_id'

In [23]:
result_rf_75_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_75['demand'].loc[prod], comp_75['pred_rf_75'].loc[prod],
                                      comp_75['SAA_pred_75'].loc[prod], cu = cu, co = co_75),2)
  result_rf_75_pscr.append(pscr)

In [24]:
RF_pooled_pscr_75 = pd.DataFrame(result_rf_75_pscr, columns=['RF_pscr_75'])
RF_pooled_pscr_75.index.name = 'item_id'

# rf for 50% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: n_estimators = 100 , max_depth = 18 , min_samples_leaf = 4

In [ ]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse')
random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 50%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [25]:
rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse',
                                    max_depth = 18, n_estimators = 100, min_samples_leaf = 4)

rf.fit(X_train, Y_train)
preds_50 = rf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/ddop/newsvendor/_WeightedNewsvendor.py:580: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model_ = model.fit(X, y)


## reorder the data




---



*   create a pandas frame 'comp_50' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [26]:
pred_50 = pd.DataFrame(preds_50, columns=['pred_rf_50'])
comp_50 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_50 = comp_50.reset_index()
comp_50['pred_rf_50'] = pred_50[['pred_rf_50']]
comp_50 = comp_50.set_index(['item_id', 'date'])

saa_50 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_50.csv').set_index(['item_id', 'date'])

comp_50 = pd.concat([comp_50, saa_50], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [27]:
result_rf_50_avc = []

for prod in prods:
  avc = round(average_costs(comp_50['demand'].loc[prod], comp_50['pred_rf_50'].loc[prod], cu = cu, co = co_50),2)
  result_rf_50_avc.append(avc)

In [28]:
RF_pooled_avc_50 = pd.DataFrame(result_rf_50_avc, columns=['RF_avc_50'])
RF_pooled_avc_50.index.name = 'item_id'

In [30]:
result_rf_50_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_50['demand'].loc[prod], comp_50['pred_rf_50'].loc[prod],
                                      comp_50['SAA_pred_50'].loc[prod], cu = cu, co = co_50),2)
  result_rf_50_pscr.append(pscr)

In [31]:
RF_pooled_pscr_50 = pd.DataFrame(result_rf_50_pscr, columns=['RF_pscr_50'])
RF_pooled_pscr_50.index.name = 'item_id'

# merge the pandas frames


---

In [32]:
RF_pooled_avc = pd.concat([RF_pooled_avc_90, RF_pooled_avc_75, RF_pooled_avc_50], axis=1)
RF_pooled_avc

,RF_avc_90,RF_avc_75,RF_avc_50
item_id,,,
0,2.17,4.75,8.35
1,0.91,1.96,3.47
2,2.20,4.63,7.70
3,1.54,3.55,6.71
4,1.90,4.18,8.29
5,1.18,2.56,4.56
6,1.62,3.70,6.79
7,1.54,3.49,6.55
8,1.11,2.48,4.64


In [33]:
RF_pooled_pscr = pd.concat([RF_pooled_pscr_90, RF_pooled_pscr_75, RF_pooled_pscr_50], axis=1)
RF_pooled_pscr

,RF_pscr_90,RF_pscr_75,RF_pscr_50
item_id,,,
0,0.42,0.37,0.34
1,0.50,0.50,0.45
2,0.24,0.17,0.17
3,0.71,0.65,0.56
4,0.41,0.32,0.26
5,0.43,0.39,0.29
6,0.51,0.48,0.43
7,0.44,0.27,0.18
8,0.43,0.33,0.28


# save the results


---

In [34]:
RF_pooled_avc.to_csv('/content/drive/MyDrive/M5/RF_pooled_avc.csv')

RF_pooled_pscr.to_csv('/content/drive/MyDrive/M5/RF_pooled_pscr.csv')